In [1]:
### This code is supposed to be run in Kaggle notebook for benchmarking the kNN approach
### https://www.kaggle.com/code/mathisjander/240709-benchmark-knn/notebook

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors


In [2]:
# define n iterations
n = 5

# load metadata

metadata = pd.read_csv('/kaggle/input/geolifeclef-2024/GLC24_PA_metadata_train.csv')

# get survey ids
survey_ids = metadata['surveyId'].unique()
survey_ids

# define train ratio
train_ratio = 0.8

# shuffle survey ids


def get_train_val_survey_ids(survey_ids, train_ratio):
    
    # shuffle survey ids
    np.random.shuffle(survey_ids)
    # split survey ids into train and val
    n_train = int(train_ratio * len(survey_ids))
    train_survey_ids = survey_ids[:n_train]
    val_survey_ids = survey_ids[n_train:]
    return train_survey_ids, val_survey_ids





In [3]:
num_classes = 11255 # Number of all unique classes within the PO and PA data.

In [4]:
import os
import torch
import tqdm
import numpy as np
import pandas as pd
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import precision_recall_fscore_support

In [5]:
from PIL import Image

def construct_patch_path(data_path, survey_id):
    """Construct the patch file path based on plot_id as './CD/AB/XXXXABCD.jpeg'"""
    path = data_path
    for d in (str(survey_id)[-2:], str(survey_id)[-4:-2]):
        path = os.path.join(path, d)

    path = os.path.join(path, f"{survey_id}.jpeg")

    return path

class BaselineTrainDataset(Dataset):
    def __init__(self, bioclim_data_dir, landsat_data_dir, sentinel_data_dir, metadata, survey_ids, transform=None):
        self.transform = transform
        self.sentinel_transform = transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5, 0.5)),
        ])
      
        self.bioclim_data_dir = bioclim_data_dir
        self.landsat_data_dir = landsat_data_dir
        self.sentinel_data_dir = sentinel_data_dir
        self.metadata = metadata
        self.metadata = self.metadata.dropna(subset="speciesId").reset_index(drop=True)
        self.metadata['speciesId'] = self.metadata['speciesId'].astype(int)
        self.label_dict = self.metadata.groupby('surveyId')['speciesId'].apply(list).to_dict()
        
        self.metadata = self.metadata.drop_duplicates(subset="surveyId").reset_index(drop=True)

        self.survey_ids = survey_ids

    def __len__(self):
        return len(self.survey_ids)

    def __getitem__(self, idx):
        
        survey_id = self.survey_ids[idx]
        
        landsat_sample = torch.nan_to_num(torch.load(os.path.join(self.landsat_data_dir, f"GLC24-PA-train-landsat-time-series_{survey_id}_cube.pt")))
        bioclim_sample = torch.nan_to_num(torch.load(os.path.join(self.bioclim_data_dir, f"GLC24-PA-train-bioclimatic_monthly_{survey_id}_cube.pt")))

        rgb_sample = np.array(Image.open(construct_patch_path(self.sentinel_data_dir, survey_id)))
        nir_sample = np.array(Image.open(construct_patch_path(self.sentinel_data_dir.replace("rgb", "nir").replace("RGB", "NIR"), survey_id)))
        sentinel_sample = np.concatenate((rgb_sample, nir_sample[...,None]), axis=2)

        species_ids = self.label_dict.get(survey_id, [])  # Get list of species IDs for the survey ID
        label = torch.zeros(num_classes)  # Initialize label tensor
        for species_id in species_ids:
            label_id = species_id
            label[label_id] = 1  # Set the corresponding class index to 1 for each species
        
        if isinstance(landsat_sample, torch.Tensor):
            landsat_sample = landsat_sample.permute(1, 2, 0)  # Change tensor shape from (C, H, W) to (H, W, C)
            landsat_sample = landsat_sample.numpy()  # Convert tensor to numpy array
            
        if isinstance(bioclim_sample, torch.Tensor):
            bioclim_sample = bioclim_sample.permute(1, 2, 0)  # Change tensor shape from (C, H, W) to (H, W, C)
            bioclim_sample = bioclim_sample.numpy()  # Convert tensor to numpy array   
        
        if self.transform:
            landsat_sample = self.transform(landsat_sample)
            bioclim_sample = self.transform(bioclim_sample)
            sentinel_sample = self.sentinel_transform(sentinel_sample)

        return landsat_sample, bioclim_sample, sentinel_sample, label, survey_id



In [6]:
# Train Dataset and DataLoader

def create_loaders(train_survey_ids, val_survey_ids):
  train_batch_size = 256
  val_batch_size = 64
  transform = transforms.Compose([
      transforms.ToTensor()
  ])

   # Load Training metadata
  train_landsat_data_path = "/kaggle/input/geolifeclef-2024/TimeSeries-Cubes/TimeSeries-Cubes/GLC24-PA-train-landsat_time_series"
  train_bioclim_data_path = "/kaggle/input/geolifeclef-2024/TimeSeries-Cubes/TimeSeries-Cubes/GLC24-PA-train-bioclimatic_monthly"
  train_sentinel_data_path="/kaggle/input/geolifeclef-2024/PA_Train_SatellitePatches_RGB/pa_train_patches_rgb"
  train_metadata_path = "/kaggle/input/geolifeclef-2024/GLC24_PA_metadata_train.csv"

  train_metadata = pd.read_csv(train_metadata_path)

  train_data = BaselineTrainDataset(train_bioclim_data_path, train_landsat_data_path, train_sentinel_data_path, train_metadata, train_survey_ids, transform=transform)
  train_loader = DataLoader(train_data, batch_size=train_batch_size, shuffle=False, num_workers=4)

  val_data = BaselineTrainDataset(train_bioclim_data_path, train_landsat_data_path, train_sentinel_data_path, train_metadata, val_survey_ids, transform=transform)
  val_loader = DataLoader(val_data, batch_size=val_batch_size, shuffle=False, num_workers=4)

  return train_loader, val_loader



In [7]:
import torch
import numpy as np
from tqdm import tqdm

def calculate_f1_score_from_tensors(y_true, y_pred, threshold=0.5):
    
    y_pred = (y_pred >= threshold)
    
    TP = (y_true & y_pred).sum(axis=1)  # True Positives per sample
    FP = (y_true & ~y_pred).sum(axis=1)  # False Positives per sample
    FN = (~y_true & y_pred).sum(axis=1)  # False Negatives per sample

    # compute f1 score for each sample
    pre = TP/(TP+FP)
    rec = TP/(TP+FN)
    f1 = 2 * pre * rec / (pre + rec)

    # Handle division by zero
    f1 = np.nan_to_num(f1)

    # compute micro-average f1 score
    micro_f1 = np.mean(f1)
    # Return mean F1 score across all samples
    return micro_f1



In [8]:
# Check if cuda is available
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
    

print(device)

cuda


In [9]:
# define model

import torch.nn.functional as F

class MultimodalEnsemble(nn.Module):
    def __init__(self):
        super(MultimodalEnsemble, self).__init__()
        
        self.landsat_norm = nn.LayerNorm([6,4,21])
        self.landsat_model = models.resnet18(weights=None)
        # Modify the first convolutional layer to accept 6 channels instead of 3
        self.landsat_model.conv1 = nn.Conv2d(6, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.landsat_model.maxpool = nn.Identity()
        
        self.bioclim_norm = nn.LayerNorm([4,19,12])
        self.bioclim_model = models.resnet18(weights=None)  
        # Modify the first convolutional layer to accept 4 channels instead of 3
        self.bioclim_model.conv1 = nn.Conv2d(4, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bioclim_model.maxpool = nn.Identity()
        
        self.sentinel_model = models.swin_t(weights="IMAGENET1K_V1")
        # Modify the first layer to accept 4 channels instead of 3
        self.sentinel_model.features[0][0] = nn.Conv2d(4, 96, kernel_size=(4, 4), stride=(4, 4))
        self.sentinel_model.head = nn.Identity()
        
        self.ln1 = nn.LayerNorm(1000)
        self.ln2 = nn.LayerNorm(1000)
        
        
    def forward(self, x, y, z):
        
        x = self.landsat_norm(x)
        x = self.landsat_model(x)
        x = self.ln1(x)
        
        y = self.bioclim_norm(y)
        y = self.bioclim_model(y)
        y = self.ln2(y)
        
        z = self.sentinel_model(z)
        
        out = torch.cat((x, y, z), dim=1)
        
        return out


In [10]:
# Training loop with custom F1 score
f1s_knn = []

for i in range(n):

    print(f"Iteration: {i+1}")

    # get train and val survey ids
    train_survey_ids, val_survey_ids = get_train_val_survey_ids(survey_ids, train_ratio)

    train_loader, val_loader = create_loaders(train_survey_ids, val_survey_ids)
    
    feature_extractor = MultimodalEnsemble().to(device)

    # Training
    print('Extracting features...')

    feature_extractor.train()
    
    # Feature Extraction
    X = []
    Y = []
    for data1, data2, data3, target , _ in tqdm(train_loader):
        data1 = data1.to(device)
        data2 = data2.to(device)
        data3 = data3.to(device)
        feature_vector = feature_extractor(data1, data2, data3).cpu().detach().numpy()

        X.append(feature_vector)
        Y.append(target)
        
    print('X:', len(X), len(X[0]), len(X[0][0]))
    print('Y:',len(X), len(X[0]), len(X[0][0]))
        
    X = np.concatenate(X, axis=0)
    Y = np.concatenate(Y, axis=0)
    
    print('X:',X.shape)
    print('X:',Y.shape)

    print('Shape of feature dataset:', )
    # instantiate a NearestNeighbors object
    nn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')

    # fit model on dataset
    print('Fitting kNN...')
    nn_model.fit(X)

    # Predict with trained models on validation set
    feature_extractor.eval()
    Y_preds = []
    Y_trues = []
    print('Validating model...')
    for data1, data2, data3, targets, _ in tqdm(val_loader):
        data1 = data1.to(device)
        data2 = data2.to(device)
        data3 = data3.to(device)
        outputs = feature_extractor(data1, data2, data3).cpu().detach().numpy()

        for output in outputs:
            distances, indices = nn_model.kneighbors(output.reshape(1, -1))
            y_neighbors = Y[indices[0], :]
            avg = y_neighbors.mean(axis=0).round()
            Y_preds.append(avg)
        Y_trues.append(targets.cpu().detach().numpy())

    Y_preds = np.vstack(Y_preds).astype(int)
    Y_trues = np.vstack(Y_trues).astype(int)
    print(Y_preds.shape, Y_trues.shape)
    


    # Calculate custom F1 score
    f1 = calculate_f1_score_from_tensors(Y_trues, Y_preds, threshold=0.5)
        
    print(f"Iteration {i+1} - Mean F1 Score: {f1}")

    # Append mean F1 score to list
    f1s_knn.append(f1)

Iteration: 1


Downloading: "https://download.pytorch.org/models/swin_t-704ceda3.pth" to /root/.cache/torch/hub/checkpoints/swin_t-704ceda3.pth
100%|██████████| 108M/108M [00:01<00:00, 66.5MB/s]


Extracting features...


100%|██████████| 279/279 [11:04<00:00,  2.38s/it]


X: 279 256 2768
Y: 279 256 2768
X: (71189, 2768)
X: (71189, 11255)
Shape of feature dataset:
Fitting kNN...
Validating model...


...

100%|██████████| 279/279 [03:07<00:00,  1.49it/s]


X: 279 256 2768
Y: 279 256 2768
X: (71189, 2768)
X: (71189, 11255)
Shape of feature dataset:
Fitting kNN...
Validating model...


100%|██████████| 279/279 [1:04:44<00:00, 13.92s/it]


(17798, 11255) (17798, 11255)
Iteration 5 - Mean F1 Score: 0.28770806844027


/tmp/ipykernel_24/614177510.py:15: RuntimeWarning: invalid value encountered in divide
  rec = TP/(TP+FN)
/tmp/ipykernel_24/614177510.py:16: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * pre * rec / (pre + rec)


In [11]:
import datetime

now = datetime.datetime.now()
timestamp = now.strftime('%Y-%m-%d_%H-%M-%S')

results = pd.DataFrame(f1s_knn, columns=['knn_f1'])
results.to_csv(f'{timestamp}_knn_benchmark_results_{n}.csv', index=False)

In [12]:
print(Y_preds.shape, Y_trues.shape)

(17798, 11255) (17798, 11255)
